In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
#import os
#import glob
import pickle
from tqdm import tqdm 
from moviepy.editor import VideoFileClip
#plt.rcParams['figure.figsize'] = (10, 6)
dist_pickle = pickle.load( open('calibration.p', 'rb') )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

def imgUndistort(img, mtx, dist):
    return cv2.undistort(img, mtx, dist, None, mtx)

def getPerspectiveTransMatrix():
    src_p = np.float32(
        [
            [688, 452],
            [1029, 670],
            [278, 670],
            [593, 452]
        ]
    )

    dst_p = np.float32(
        [
            [1280-380, 0],
            [1280-380, 720],
            [380, 720],
            [380, 0]
        ]
    )
    M = cv2.getPerspectiveTransform(src_p, dst_p)
    Minv = cv2.getPerspectiveTransform(dst_p, src_p)
    return M, Minv

def perspectiveTrans(img, M):
    h,w = img.shape[:2]
    return cv2.warpPerspective(img, M, (w,h), flags=cv2.INTER_LINEAR)

def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        # Apply x or y gradient with the OpenCV Sobel() function
        # and take the absolute value
        if orient == 'x':
            abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
        if orient == 'y':
            abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
        # Rescale back to 8 bit integer
        scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
        # Create a copy and apply the threshold
        binary_output = np.zeros_like(scaled_sobel)
        # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
        binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

        # Return the result
        return binary_output
    
def mag_thresh(img, sobel_kernel=3, thresh=(0, 255)):
    """
    Returns an array of the same size as the input image of ones where gradients
    were in the threshold range, and zeros everywhere else.
    :param img: input image in BGR format.
    :param sobel_kernel: size of the sobel kernel to apply (must be odd number >= 3).
    :param thresh: threshold (0 to 255) for determining which gradients to include when creating binary output.
    """
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)/4
    # Calculate the magnitude of the gradient
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Scale to 8-bit (0-255) then convert to type = np.uint8
    scaled_gradmag = np.uint8(255*gradmag/np.max(gradmag))
    # Create a mask of 1's where the scaled gradient magnitude is within the given thresholds
    mag_binary = np.zeros_like(scaled_gradmag)
    mag_binary[(scaled_gradmag >= thresh[0]) & (scaled_gradmag <= thresh[1])] = 1
    # Return binary output image
    return mag_binary


def dir_thresh(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    """
    Returns a binary image of the same size as the input image of ones where gradient directions
    were in the threshold range, and zeros everywhere else.
    :param img: input image in BGR format.
    :param sobel_kernel: size of the sobel kernel to apply (must be odd number >= 3).
    :param thresh: threshold (0 to pi/2) for determining which gradients to include when creating binary output.
    """
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Use np.arctan2(abs_sobely, abs_sobelx) to calculate direction of gradient
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    # Create a mask of 1's where the gradient direction is within the given thresholds
    dir_binary = np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    # Return binary output image
    return dir_binary


def color_thresh(img, r_thresh=(170, 255), s_thresh=(20, 100)):
    """
    Returns a binary image of the same size as the input image of ones where pixel values
    were in the threshold range, and zeros everywhere else.
    :param img: input image in BGR format.
    :param r_thresh: threshold (0 to 255) for determining which pixels from r_channel to include in binary output.
    :param s_thresh: threshold (0 to 255) for determining which pixels from s_channel to include in binary output.
    """
    # Apply a threshold to the R channel
    r_channel = img[:,:,2]
    r_binary = np.zeros_like(img[:,:,0])
    # Create a mask of 1's where pixel value is within the given thresholds
    r_binary[(r_channel > r_thresh[0]) & (r_channel <= r_thresh[1])] = 1

    # Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    # Apply a threshold to the S channel
    s_channel = hls[:,:,2]
    s_binary = np.zeros_like(s_channel)
    # Create a mask of 1's where pixel value is within the given thresholds
    s_binary[(s_channel > s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

    # Combine two channels
    combined = np.zeros_like(img[:,:,0])
    combined[(s_binary == 1) & (r_binary == 1)] = 1
    #combined[(r_binary == 1)] = 1
    # Return binary output image
    return combined

def combineThreshold(RGB_img):
    abs_binary = abs_sobel_thresh(RGB_img, orient='x', thresh_min=60, thresh_max=255)
    mag_binary = mag_thresh(RGB_img, sobel_kernel=25, thresh=(65, 255))
    dir_binary = dir_thresh(RGB_img, sobel_kernel=15, thresh=(0.7, 1.3))
    grad_binary = np.zeros_like(RGB_img[:,:,0])
    grad_binary[(((abs_binary == 1) | (mag_binary == 1)) & (dir_binary == 0))] = 1
    color_binary = color_thresh(RGB_img, r_thresh=(60, 255), s_thresh=(80, 255))
    combo_binary = np.zeros_like(RGB_img[:,:,0])
    combo_binary[(grad_binary == 1) | (color_binary == 1)] = 255
    return combo_binary

def findLaneLineWithBirdView(binary_warped):
    
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)
    position_weights = [-(x-binary_warped.shape[1]/2)**2/25 for x in range(histogram.shape[0])]
    histogram = histogram + position_weights
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 80
    # Set minimum number of pixels found to recenter window
    minpix = 60
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    #out_img[left_lane_inds, left_lane_inds] = [255, 0, 0]
    #out_img[right_lane_inds, right_lane_inds] = [0, 0, 255]

    #binary_warped = thres_img
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    #out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    #out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    xpts = np.dstack((left_fitx, ploty)).reshape((720,2)).astype(np.int32)
    ypts = np.dstack((right_fitx, ploty)).reshape((720,2)).astype(np.int32)
    cv2.polylines(out_img, [xpts], False, (0,255,0), 2)
    cv2.polylines(out_img, [ypts], False, (0,255,0), 2)
    return out_img,leftx, lefty, left_fit, rightx, righty, right_fit

def maskLaneLineHUD(undist, thres_img, Minv):
    birdview, leftx, lefty, left_fit, rightx, righty, right_fit = findLaneLineWithBirdView(thres_img)
    binary_warped = thres_img
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    y_eval = np.max(ploty)
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(thres_img).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (undist.shape[1], undist.shape[0]))
    
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    # Draw top-right overlay
    mini_birdview = np.zeros((int(birdview.shape[0]/3), int(birdview.shape[1]/3)))
    mini_birdview = cv2.resize(birdview, (mini_birdview.shape[1],mini_birdview.shape[0]))
    #result[:mini_birdview.shape[0],-mini_birdview.shape[1]:,:] = mini_birdview
    result[:mini_birdview.shape[0],-mini_birdview.shape[1]:,:] \
    = cv2.addWeighted(result[:mini_birdview.shape[0],-mini_birdview.shape[1]:,:], 1, mini_birdview, 0.9, 0)
    
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    #print((ploty*ym_per_pix).shape, (leftx*xm_per_pix).shape)
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radius of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    #print(left_curverad, 'm', right_curverad, 'm')
    
    # Now finding road center
    h = undist.shape[0]
    w = undist.shape[1]
    l_fit_x_int = left_fit[0]*h**2 + left_fit[1]*h + left_fit[2]
    r_fit_x_int = right_fit[0]*h**2 + right_fit[1]*h + right_fit[2]
    lane_center = (r_fit_x_int + l_fit_x_int) /2
    
    bias = (w/2 - lane_center) * xm_per_pix
    if bias > 0:
        hud_center = "Vehicle is %.2f (m) right of center" % np.abs(bias)
    else:
        hud_center = "Vehicle is %.2f (m) left of center" % np.abs(bias)
    
    #radius = np.mean([left_curverad, right_curverad])
    hud_lradius = "Radius of left curvature =  %.2f (m)" % left_curverad
    hud_rradius = "Radius of right curvature =  %.2f (m)" % right_curverad
    
    font = cv2.FONT_HERSHEY_COMPLEX
    cv2.putText(result,hud_center,(10,40), font, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(result,hud_lradius,(10,100), font, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(result,hud_rradius,(10,160), font, 1,(255,255,255),2,cv2.LINE_AA)
    return result

def drawLaneLine(img):
    test_img_undist = imgUndistort(img, mtx, dist)
    M, Minv = getPerspectiveTransMatrix()
    test_img_pertrans = perspectiveTrans(test_img_undist, M)
    thres_img = combineThreshold(test_img_pertrans)
    vis_img = maskLaneLineHUD(test_img_undist, thres_img, Minv)
    
    return vis_img


In [4]:
out_dir='./'
output = out_dir+'processed_project_video.mp4'
clip = VideoFileClip("project_video.mp4")
out_clip = clip.fl_image(drawLaneLine)
%time out_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video ./processed_project_video.mp4
[MoviePy] Writing video ./processed_project_video.mp4


100%|█████████▉| 1260/1261 [03:28<00:00,  5.93it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./processed_project_video.mp4 

CPU times: user 4min 20s, sys: 15.5 s, total: 4min 35s
Wall time: 3min 28s


In [6]:
%%HTML
<video width="960" height="540" controls>
  <source src="processed_project_video.mp4" type="video/mp4">
</video>